# test

In [1]:
pip install pytest


In [2]:
!git clone https://github.com/MohamedMassaoudi-code/cybermicrogrid.git
%cd cybermicrogrid
!pip install -e .


Cloning into 'cybermicrogrid'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 64 (delta 30), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 34.36 KiB | 533.00 KiB/s, done.
Resolving deltas: 100% (30/30), done.
/content/cybermicrogrid
Obtaining file:///content/cybermicrogrid
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import cybermicrogrid

In [4]:
# tests/test_library.py

import pytest
from torch_geometric.data import Data

# Import the relevant modules from your library
from cybermicrogrid.networks import create_campus_microgrid_network
from cybermicrogrid.utils import ensure_features
from cybermicrogrid.environment import PowerGridEnvironment
from cybermicrogrid.agents import DummyAgent
from cybermicrogrid.training import train_value_based_agent


In [6]:
# tests/test_library.py

import pytest
from torch_geometric.data import Data

# Import the relevant modules from your library
from cybermicrogrid.networks import create_campus_microgrid_network
from cybermicrogrid.utils import ensure_features
from cybermicrogrid.environment import PowerGridEnvironment
from cybermicrogrid.agents import DummyAgent
from cybermicrogrid.training import train_value_based_agent

@pytest.fixture
def setup_environment():
    """Creates a simple PowerGridEnvironment with default (none) attack mode."""
    net = create_campus_microgrid_network()
    nbuses = net.bus.shape[0]
    data = Data(num_nodes=nbuses)
    data = ensure_features(data)
    env = PowerGridEnvironment(pp_net=net, data=data, attack_mode="none", attack_strength=0.1)
    return env

def test_environment_step(setup_environment):
    """Tests that the environment can reset and step without errors."""
    env = setup_environment
    state = env.reset()
    assert state is not None, "Environment reset returned None."

    action = 0  # Toggle the first line
    new_state, reward, done = env.step(action)
    assert new_state is not None, "Environment step returned a None state."
    assert isinstance(reward, float), "Reward should be a float."
    assert done in [True, False], "Done should be a boolean value."

def test_dummy_agent_training(setup_environment):
    """
    Tests that a DummyAgent can train (briefly) without raising errors.
    We'll do a very short run to ensure the code is working.
    """
    env = setup_environment
    net = env.pp_net
    data = env.current_data

    # The number of buses is also our naive 'state dimension' for a DQN-based approach
    state_dim = data.num_nodes
    action_dim = len(env.action_space)

    agent = DummyAgent()
    # Perform a single-episode training to confirm it runs
    train_value_based_agent(net, data, agent, num_episodes=1, attack_mode="none", attack_strength=0.0)

    # If we reached this point, it means training did not raise an exception.
    assert True, "DummyAgent training ran successfully."


In [7]:
# Import the environment module from the cybermicrogrid library
from cybermicrogrid.environment import PowerGridEnvironment

# Define a list of supported attack modes (as documented in PowerGridEnvironment)
attack_modes = [
    "NONE", "FDI", "MITM", "DOS", "REPLAY", "DDOS",
    "APT", "RANSOMWARE", "PHISHING", "ZERO_DAY", "INSIDER",
    "GPS_SPOOF", "SUPPLY_CHAIN", "CONTROL_HIJACK", "CASCADING",
    "SENSOR_JAM", "DATA_EXFIL", "MALWARE"
]

# Print the supported cyberattack modes
print("Supported Cyberattack Modes:")
for mode in attack_modes:
    print("-", mode)


Supported Cyberattack Modes:
- NONE
- FDI
- MITM
- DOS
- REPLAY
- DDOS
- APT
- RANSOMWARE
- PHISHING
- ZERO_DAY
- INSIDER
- GPS_SPOOF
- SUPPLY_CHAIN
- CONTROL_HIJACK
- CASCADING
- SENSOR_JAM
- DATA_EXFIL
- MALWARE
